## CSV pour GEPHI :
- ~~utiliser co-occu mots mais aussi juste mots/hashtags/mentions présents dans un même tweet~~
- ~~plusieurs graphes : un avec tous les types, un avec seulement les personnes, un avec seulement #, un @~~

In [2]:
import json,glob
import nltk
from nltk.tokenize import word_tokenize
import csv
from datetime import datetime
import tqdm
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
import spacy
nlp = spacy.load('fr_core_news_sm')

def ouvrir_json(chemin):
    with open(chemin, encoding="utf-8") as f:
        toto = json.load(f)
    return toto


/home/fanny/anaconda3/envs/main/lib/python3.10/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
"""V2 AVEC LEMMATISATION ET FILTRE : bcp + long"""
version = "lem_filtree"
d = datetime.now()
date = str(d.day)+"-0"+str(d.month)
filtre_min = 50 #nb occu min qu'un noeud doit avoir pour être mis dans le CSV
    
"""Créer les structures de données pour remplir ensuite les CSV : un fichier de noeuds avec id, label, type, weight (nb_occu)
et un fichier de relations avec source, target, weight"""

co_o = []
id_mot = {}
k = 0
n = []

for doc in tqdm.tqdm(glob.glob("corpus_projet_plus/*")):
    corpus = ouvrir_json(doc)
    data = corpus["data"]
    for n_tweet, tweet in enumerate(data):
        text = tweet["text"].lower()

        text_tok = text.lower().split()
        #text_tok = [tok for tok in text_tok if (tok.isalpha() or tok.startswith("@") or tok.startswith('#'))] #and tok not in ".,;!?👉"] #and tok.isalpha() mais perd @ et #
        text_tok = ["hashtag"+tok if tok.startswith('#') else tok for tok in text_tok] #astuce sinon lemmatisation enlève les dièses donc on perd notion = à enlever plus tard avec regex de hashtag
        text_lem = nlp(" ".join(text_tok))
        lem = [str(tok.lemma_) for tok in text_lem if tok.lemma_ not in fr_stop and (tok.lemma_.isalpha() or tok.lemma_.startswith("@") or tok.lemma_.startswith('hashtag#'))]
        #filtrer mots vides (ici, pas avant, sinon peut biaiser lemmatisation) + ponctuation

        
        """Pour le fichier de noeuds : id (nb incrémenté), type mot, nb_occu (poids)"""
        for token in lem:
            #toutes les lignes qui sont de cette forme servent à regrouper les hashtags type #reformedesretraites (avec/sans s, accent, etc)
            if ("reforme" in token or "réforme" in token) and "retraite" in token:
                token = "hashtag#reformedesretraites"
            if token not in id_mot:
                nb_occu = 1
                type_mot = "mot" 
                if token.startswith("@"):
                    type_mot = "personne"
                if token.startswith("hashtag#"):
                    type_mot = "hashtag"
                id_mot[token] = [k, type_mot, nb_occu]
                k+=1
            else:
                id_mot[token][2]+=1
        
        """Pour le fichier de relations : toutes les combinaisons possibles dans le tweet (lier tous mots d'un même tweet)
        mais compter qu'une seule fois la relation par tweet"""       
        for i in range(len(lem)):
            mot = lem[i]
            if ("reforme" in mot or "réforme" in mot) and "retraite" in mot:
                mot = "hashtag#reformedesretraites"
            if i < len(lem)-1 and mot not in fr_stop and (mot.isalpha() or mot.startswith("@") or mot.startswith('#') or mot.startswith('hashtag#')):
                for j in range(len(lem)+1):
                    if j < len(lem)-2:
                        if j == i:
                            j+=1
                        if ("reforme" in lem[j] or "réforme" in lem[j]) and "retraite" in lem[j]:
                            mot_svt = "hashtag#reformedesretraites"
                        else:
                            mot_svt = lem[j]
                        co_o.append(sorted([mot,mot_svt]))
                        #if sorted([mot,mot_svt]) not in co_o or n_tweet not in n:
                            #co_o.append(sorted([mot,mot_svt]))
                            #n.append(n_tweet) #pour vérifier
    
"""Pour ne pas avoir à relancer toute la lemmatisation etc"""

with open('id_mot.json', 'w') as f:
    json.dump(id_mot, f, indent=4)
with open('co_o.json', 'w') as f:
    json.dump(co_o, f, indent=4)

100%|████████████████████████████████████████| 82/82 [2:47:44<00:00, 122.74s/it]


In [ ]:
exclus = []   
"""Écrire le fichier de noeuds"""
with open(f'csv/noeuds_{date}_{version}.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["Id","Label","Kind","Weight"])
    for mot, liste_att in id_mot.items(): #dico forme {mot : [id, type, nb_occu]}
        if liste_att[1] == "mot": #filtre + strict sur les mots
            filtre=150
        else:
            filtre= filtre_min
        if liste_att[2] > filtre:
            if mot.startswith("hashtag#"): #régler le trick utilisé pour contourner la lemmatisation des hashtags
                mot = mot.split("hashtag")[-1]
            writer.writerow([liste_att[0],mot,liste_att[1],liste_att[2]])
        else: #mettre de côté les id de noeuds filtrés pour pas les prendre dans les relations
            exclus.append(liste_att[0])
            
"""Remplacer les mots par leur id pour le fichier de relations et compter le nombre de co-occurrences pour poids"""
from collections import Counter
counter = Counter(tuple(x) for x in co_o)
id_o = [[id_mot[k[0]][0], id_mot[k[1]][0], v] for k,v in counter.items()]
#id_o[:10]   

with open(f'csv/liens_{date}_{version}.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["Source","Target","Weight"])
    for liste in id_o: #liste de listes type [source, target, weight]
        if liste[0] not in exclus and liste[1] not in exclus:
            writer.writerow([liste[0], liste[1],liste[2]])

In [48]:
"""Filtrer fichiers de noeuds et relations pour faire d'autres graphiques zoomés : un pour les personnes,
un pour les hashtags, un pour les mots"""
personne = []
id_p = []
hashtag = []
id_h = []
mot = []
id_m = []

with open(f'../csv/noeuds_{date}_lem_filtree.csv', 'r') as f:
    reader = csv.reader(f)
    for i,row in enumerate(reader):
        if i>0:
            if row[2]=="personne":
                personne.append(row) #pour pouvoir remplir le csv noeuds personne ensuite
                id_p.append(row[0]) #pour pouvoir remplir le csv liens personne ensuite (avec seulement noeuds de source/sortie inclus)
            if row[2]=="hashtag":
                hashtag.append(row)
                id_h.append(row[0])
            if row[2]=="mot":
                mot.append(row)
                id_m.append(row[0])

#liste de listes contenant les lignes à garder pour les nvx fichiers de relations
liens_p = []
liens_h = []
liens_m = []
with open(f'../csv/liens_{date}_lem_filtree.csv', 'r') as f:
    reader = csv.reader(f)
    for i, row in enumerate(reader):
        if i>0:
            if row[0] in id_m and row[1] in id_m:
                liens_m.append(row)
            if row[0] in id_h and row[1] in id_h:
                liens_h.append(row)
            if row[0] in id_p and row[1] in id_p:
                liens_p.append(row)
                
#écrire nvx csv
kind = ["mots","personnes","hashtags"]
liens_kind = [liens_m,liens_p,liens_h]
noeuds_kind = [mot,personne,hashtag]
for i, k in enumerate(kind):
    with open(f'../csv/liens_{date}_{k}.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["Source","Target","Weight"])
        for row in liens_kind[i]:
            writer.writerow(row)
    with open(f'../csv/noeuds_{date}_{k}.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["Id","Label","Kind","Weight"])
        for row in noeuds_kind[i]:
            writer.writerow(row)

In [42]:
"""Seulement sur users à + de 100k followers, pour mentions = graphe avec direction !!!"""
version = "populaire"
from datetime import datetime
import tqdm
import csv
d = datetime.now()
date = str(d.day)+"-0"+str(d.month)
filtre_min = 1 #nb occu min qu'un noeud doit avoir pour être mis dans le CSV

k = 0
popular = {} #dico type {user:[id]} pour pouvoir ensuite seulement garder tweets de ces gens
pop_mh = {} #pour garder les id des mentions et hashtags utilisés par un utilisateur
poids = {} #pour donner poids aux noeuds selon nb followers (pour users)
id_mot = {}

for doc in tqdm.tqdm(glob.glob("../corpus/*")):
    corpus = ouvrir_json(doc)
    data = corpus["data"]
    includes = corpus["includes"]
    
    """PARTIE INCLUDES/USERS"""
    for user in includes["users"]:
        if user['public_metrics']['followers_count'] > 100000:
            popular[user['id']]=user['username']
            poids[user['id']]= user['public_metrics']['followers_count']
                
    for n_tweet, tweet in enumerate(data):
        if tweet["author_id"] in popular.keys():
            text = tweet["text"]
            text = text.replace(",.-;:?!)(","")
            text = text.replace(",","")

            text_tok = text.split()
            
            #garder seulement mentions (et hashtags)
            text_tok = [tok for tok in text_tok if tok.startswith("@")] #or tok.startswith('#')] #and tok not in ".,;!?👉"] #and tok.isalpha() mais perd @ et #

            """Pour le fichier de noeuds : id (nb incrémenté), type mot, nb_occu (poids)"""
            for token in text_tok:
                if token not in id_mot:
                    nb_occu = 1
                    id_mot[token] = [k, type_mot, nb_occu]
                    k+=1
                else:
                    id_mot[token][2]+=1

                #ne pas créer de nv identifiant si la personne mentionnée fait partie des users sélectionnés
                if token.split('@')[-1] in popular.values():
                    key = list(filter(lambda x: popular[x] == token.split('@')[-1], popular))[0]
                if tweet['author_id'] not in pop_mh:
                    #dico de dico {userid : {id_mention : nb_occu_mention}}
                    pop_mh[tweet['author_id']] = {}
                if token.split('@')[-1] in popular.values():
                    pop_mh[tweet['author_id']][int(key)] = pop_mh[tweet['author_id']].get(int(key),0)+1
                else:
                    pop_mh[tweet['author_id']][k] = pop_mh[tweet['author_id']].get(k,0)+1
                    
                    
#Écrire le fichier de noeuds en ajoutant les utilisateurs aussi
with open(f'../csv/noeuds_{version}.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["Id","Label","Kind","Weight"])
    for mot, liste_att in id_mot.items(): #dico forme {mot : [id, type, nb_occu]}
        writer.writerow([liste_att[0],mot,liste_att[1],liste_att[2]])
    for idd, username in popular.items():#ajouter les utilisateurs populaires (comme personne mais avec poids à 100)
        writer.writerow([idd, "@"+username, "personne", round(poids[idd]/10000)])
            

with open(f'../csv/liens_{version}.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["Source","Target","Type","Weight"])
    for userid, liste_m in pop_mh.items(): 
        for m,nb in liste_m.items():
            writer.writerow([userid, m, "Directed", nb])

100%|██████████████████████████████████████████| 82/82 [00:00<00:00, 257.58it/s]


In [27]:
"""OBSOLÈTE
V1 SANS LEMMATISATION NI FILTRE"""
"""Créer les structures de données pour remplir ensuite les CSV : un fichier de noeuds avec id, label, type, weight (nb_occu)
et un fichier de relations avec source, target, weight"""

version = "filtre"
co_o = []
id_mot = {}
k = 0
n = []

for doc in tqdm.tqdm(glob.glob("corpus_projet_plus/*")):
    corpus = ouvrir_json(doc)
    data = corpus["data"]
    for n_tweet, tweet in enumerate(data):
        text = tweet["text"].lower()

        text_tok = text.lower().split()
        text_tok = [tok for tok in text_tok if tok not in fr_stop and (tok.isalpha() or tok.startswith("@") or tok.startswith('#'))] #and tok not in ".,;!?👉"] #and tok.isalpha() mais perd @ et #

        """Pour le fichier de noeuds : id (nb incrémenté), type mot, nb_occu (poids)"""
        for token in text_tok:
            if token not in id_mot:
                nb_occu = 1
                type_mot = "mot" 
                if token.startswith("@"):
                    type_mot = "personne"
                if token.startswith("#"):
                    type_mot = "hashtag"
                id_mot[token] = [k, type_mot, nb_occu]
                k+=1
            else:
                id_mot[token][2]+=1
        
        """Pour le fichier de relations : toutes les combinaisons possibles dans le tweet (lier tous mots d'un même tweet)
        mais compter qu'une seule fois la relation par tweet"""       
        for i in range(len(text_tok)):
            mot = text_tok[i]
            if i < len(text_tok)-1 and mot not in fr_stop and (mot.isalpha() or mot.startswith("@") or mot.startswith('#')):
                for j in range(len(text_tok)+1):
                    if j < len(text_tok)-2:
                        if j == i:
                            j+=1
                        if sorted([mot,text_tok[j]]) not in co_o or n_tweet not in n:
                            co_o.append(sorted([mot,text_tok[j]]))
                            n.append(n_tweet) #pour vérifier

"""Écrire le fichier de noeuds"""
exclus = []
with open(f'csv/noeuds_2301_{version}.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["Id","Label","Kind","Weight"])
    for mot, liste_att in id_mot.items(): #dico forme {mot : [id, type, nb_occu]}
        if liste_att[2] > 10:
            writer.writerow([liste_att[0],mot,liste_att[1],liste_att[2]])
        else: #mettre de côté les id de noeuds pas sauvegardés pour pas les prendre dans les relations
            exclus.append(liste_att[0])
        
"""Remplacer les mots par leur id pour le fichier de relations"""
id_o = []
for k in co_o: #pour les poids
    ligne = []
    for mot in k:
        ligne.append(id_mot[mot][0])
    ligne.append(co_o.count(k))
    id_o.append(ligne)

with open(f'csv/liens_2301_{version}.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["Source","Target","Weight"])
    for liste in id_o: #liste de listes type [source, target, weight]
         if liste[0] not in exclus and liste[1] not in exclus:
            writer.writerow([liste[0], liste[1],liste[2]])

100%|███████████████████████████████████████████| 23/23 [06:30<00:00, 16.99s/it]
